# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## symplectic generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


## Random Word generator

In [5]:
def word_generator(length=5)
  rstr = ''
  length.times{|k| rstr += Rseq.sample.to_s}
  return Word.new(rstr)
end
def wordgen(length=5)
  word_generator(length)
end;nil

# Goldman bracket & Turaev cobracket

## Partition class, Term class and Expansion class

In [621]:
class Partition < Hash
  def initialize(word: Word.new, div: 0)
    self[:word] = word
    self[:div] = div
  end
end

#----------------------------------
class Term < Hash
  '''
    A Hash with keys :coeff, :words and :divs
  '''
  def initialize(coeff: 0, words: [], divs: [])
    begin      
      self[:coeff] = coeff
      self[:words] = words
    rescue => e
      p e.backtrace
    end
  end
  
  def cyclic_reduce
    trm = self.class.new(coeff: self[:coeff], divs: self[:divs])
    trm[:words] = self[:words].map(&:cyclic_reduce)
    return trm
  end
  
  def show
    coeff = (self[:coeff] == 1) ? '' : "(#{self[:coeff]})"
    body = self[:words].map(&:show).join("\u{2227}")
    coeff + body
  end
  
  def equiv?(a_term)
    '''
      return 1(match), -1(reversely match) and 0(not match).
    '''
    (self[:words].size).times do |k|
      flags = self[:words].zip(a_term[:words].rotate(k)).map{|pair| pair[0].conjugate?(pair[1])}
      if flags.all?{|tf| tf}
        return (-1)**k
        break
      end
    end
    return 0
  end
end

#-----------------------------------
class Expansion < Array
  '''
    An Array of Terms
  '''
  def show(simplify_level=0)
    expn = case simplify_level
      when 1
        self.delete_if{|t| t[:coeff] == 0}
      when 2
        self.simplify
      else 
        self.rotate(0)
      end
    #---
    mstr = expn.map{|term| term.show}.join(' + ')
    return (mstr.empty?) ? '0' : mstr
  end

  def simplify
    expn = self.class.new
    monomials = self.map{|mono| mono.cyclic_reduce unless mono[:coeff] == 0}.compact
    until monomials.size == 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      #---
      monomials.each do |mono|
        switch = mono.equiv?(pop)
        unless switch == 0
          pop[:coeff] += mono[:coeff] * switch
          pop[:divs] << mono[:divs]
          mono[:coeff] = nil
        end
      end
      expn << pop if pop[:coeff] != 0
      monomials.delete_if{|mono| mono[:coeff].nil?}
    end
    return (expn.empty?) ? (self.class.new << Term.new) : expn
  end
end; nil

### scratch

In [372]:
terms = [
  Term.new,
  Term.new(words: [wordgen(8)], coeff: -1), 
  Term.new(words: [wordgen(3), wordgen(2)], coeff: 1)
]

terms.each{|t| printf t.show + ' |-reduce-> ' + t.cyclic_reduce.show + "\n" }
nil

(0) |-reduce-> (0)
(-1)aaDEdeFf |-reduce-> (-1)aaDEde
FFf∧dB |-reduce-> F∧dB


In [352]:
count = 10
count.times do |k|
  expn = Expansion.new() 
  5.times do |i|
    cf = ( (-1)**(i.modulo(3)) - (-1)**((i+1).modulo(3)) )/2
    ws = [3,3].map{|k| word_generator(k)}
    expn << Term.new(divs: '', coeff: cf, words: ws)
  end
  if expn.simplify.size < 4
    break
  end
end; nil

## Linking of a pair of divisions

In [620]:
def lk(ptn1, ptn2)
  '''
    Input: Patitions {word: a Word, div: an pos. Integer}
  '''
  hashes = [ptn1, ptn2].map do |ptn|
    w = ptn[:word].flatten
    k = ptn[:div]
    s = w.size
    [
      {index: ( k%s ) + 1, letter: w[ k%s ], sign: 1}, 
      {index: k, letter: w[k-1].inverse, sign: -1}
    ]
  end  
  total = [0,1].product([0,1]).map{|i,j| epsilon(hashes[0][i], hashes[1][j])}.sum
  return (-1)*(total/2)
end

#--------------------------------------------
def epsilon(h1, h2)
  '''
    Input: Hashes {index: a pos. Integer, letter: a Letter, sign: 1 or -1}
  '''
  result = h1[:sign]*h2[:sign]
  result *= unless (h1[:letter].show == h2[:letter].show)
    (Rseq.index(h1[:letter]) < Rseq.index(h2[:letter])) ? 1 : -1
  else
    (h1[:index] < h2[:index]) ? h2[:sign] : h1[:sign]*(-1)
  end
  return result
end; nil

## Goldman bracket

In [505]:
def bracket(w1, w2)
  """
  retern: an Expansion (an Array of Terms {coeff: *, words: *, divs: *})
  """
  #-----
  expn = Expansion.new
  unless (w1.show == '1' || w2.show == '1')
    ws = [w1, w2]
    (w1.size.times.to_a).product(w2.size.times.to_a).each do |inds|
      ptns = [0,1].map{|k| Partition.new(word: ws[k], div: inds[k]+1)}
      cpws = [0,1].map{|k| ws[k].cyclic_permutation(inds[k]+1)}
      expn << Term.new(divs: ptns.map{|p| p[:div]}, coeff: lk(*ptns), words: [cpws[0]*cpws[1]])
    end
  end
  return expn
end; nil

### scratch

In [522]:
# w1 = Word.new(a[1], a[2], b[1].inverse)
# w2 = Word.new(b[2], a[1], a[1].inverse) #, b[2])
w1, w2 = wordgen(2), wordgen(3)
printf [w1, w2].map(&:show).join(', ')  + "\n-------\n"

bra = bracket(w1, w2)

bra.each do |t|
  t.each{|k,v| printf "#{k}: " + ((k == :words) ? v[0].show : v.to_s) + ", "}
  printf "\n---\n"
end
puts bra.show, "= " + bra.show(2)
nil

aB, bFa
-------
coeff: 0, words: Ba.Fab, 
---
coeff: 0, words: Ba.abF, 
---
coeff: 1, words: Ba.bFa, 
---
coeff: 1, words: aB.Fab, 
---
coeff: -1, words: aB.abF, 
---
coeff: 1, words: aB.bFa, 
---
(0)Ba.Fab + (0)Ba.abF + Ba.bFa + aB.Fab + (-1)aB.abF + aB.bFa
= aFa + aBFab


## Turaev cobraket

In [507]:
def cobracket(myw, verbose: false)
  """
  return: an Expansion (an Array of Terms {coeff: *, words: *, divs: *})
  """
  expn = Expansion.new
  unless myw.show == '1' 
    divs_combis = (1..myw.size).to_a.combination(2)  #2つの div の組み合わせの全体
    divs_combis.each do |combi|
      #--- the pair of partitions ---
      ptns = combi.map{|k| Partition.new(word: myw, div: k)} 
      #--- divide the word --- 
      facs = myw.flatten.factors
      words = Array.new(2)
      words[0] = Word.new(facs[0..combi[0]-1]+facs[combi[1]..-1])
      words[1] = Word.new(facs[combi[0]..combi[1]-1])    
        #[TODO] 上の2つの Word の定義に cyclic_permutation を使えないか?
      expn << Term.new(divs: combi, coeff: lk(*ptns), words: words)
    end
  end
  #-----
  return expn
end; nil

### scratch

In [519]:
#mw = Word.new(b[1], a[1], b[1]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
mw = wordgen(5)
printf mw.show + "\n-------\n"

cobra = cobracket(mw, verbose: false) #true) #

# cobra.each do |t|
#   t.each do |k,v| 
#     p "#{k}: " + ((k == :words) ? v[0].show : v.to_s)
#   end
#   printf "---\n"
# end

puts cobra.show, "= " + cobra.show(2)
nil

cDacd
-------
(0)cacd∧D + (0)ccd∧Da + cd∧Dac + (0)c∧Dacd + (0)cDcd∧a + cDd∧ac + (0)cD∧acd + (0)cDad∧c + cDa∧cd + (0)cDac∧d
= c∧ac


# Conditions

## Jacobi Identity

$$\nabla\circ(\nabla\otimes 1)\circ N = 0$$

In [535]:
three_words = Array.new(3)
max_length = 3

three_words.map!{|w| word_generator(rand(1..max_length))}
puts three_words.map(&:show).join(" \u{2297} ") + "  |---> " + "\n------"

total = Expansion.new
3.times do |k|
  bra = bracket(three_words[0],three_words[1])
  puts "\t\u{2207}(" + three_words[0..1].map(&:show).join(" \u{2297} ") + ") \u{2297} " + three_words[2].show + " ="
  bra.each do |h|
    bra2 = bracket(h[:words][0], three_words[2])
    puts "\t\t+ (#{bra2.show(1)}) \u{2297} #{three_words[2].show}"
    bra2.each do |hh|
      hh[:coeff] *= h[:coeff]
      total << hh
    end
  end
  three_words.rotate!
end

printf "---------\n"
p total.show(1)
printf "---------\n"
p total.show(2)
nil

A ⊗ Aef ⊗ aCD  |---> 
------
	∇(A ⊗ Aef) ⊗ aCD =
		+ ((-1)AAef.CDa + AAef.aCD + (-1)AefA.CDa + AefA.aCD) ⊗ aCD
		+ ((-1)AeAf.CDa + AeAf.aCD + (-1)AfAe.CDa + AfAe.aCD) ⊗ aCD
		+ ((-1)AefA.CDa + AefA.aCD + (-1)AAef.CDa + AAef.aCD) ⊗ aCD
	∇(Aef ⊗ aCD) ⊗ A =
		+ ((-1)aefACD.A + efACDa.A) ⊗ A
		+ ((-1)aCefAD.A + CefADa.A) ⊗ A
		+ ((-1)aCDefA.A + CDefAa.A) ⊗ A
		+ ((-1)afAeCD.A + fAeCDa.A) ⊗ A
		+ ((-1)aCfAeD.A + CfAeDa.A) ⊗ A
		+ ((-1)aCDfAe.A + CDfAea.A) ⊗ A
		+ ((-1)aAefCD.A + AefCDa.A) ⊗ A
		+ ((-1)aCAefD.A + CAefDa.A) ⊗ A
		+ ((-1)aCDAef.A + CDAefa.A) ⊗ A
	∇(aCD ⊗ A) ⊗ Aef =
		+ ((-1)aACD.Aef + ACDa.Aef + (-1)CDaA.efA + CDaA.Aef) ⊗ Aef
		+ ((-1)aCAD.Aef + CADa.Aef + (-1)DaCA.efA + DaCA.Aef) ⊗ Aef
		+ (CDAa.Aef + (-1)aCDA.efA) ⊗ Aef
---------
"AAef.CDa + (-1)AAef.aCD + AefA.CDa + (-1)AefA.aCD + (-1)AefA.CDa + AefA.aCD + (-1)AAef.CDa + AAef.aCD + aAefCD.A + (-1)AefCDa.A + (-1)aCDAef.A + CDAefa.A + (-1)aACD.Aef + ACDa.Aef + (-1)CDaA.efA + CDaA.Aef + (-1)CDAa.Aef + aCDA.efA"
---------
"(0)"

In [541]:
three_words = Array.new(3)
max_length = 3
count = 0 #500

count.times do |k|
  three_words.map!{|w| word_generator(rand(1..max_length))}
  total = Expansion.new
  3.times do |k|
    bra = bracket(three_words[0],three_words[1])
#     puts "\u{2207}(" + three_words[0..1].map(&:show).join(" \u{2297} ") + ") \u{2297} " + three_words[2].show + " ="
    bra.each do |h|
      bra2 = bracket(h[:words][0], three_words[2])
#       puts "+ (#{bra2.show}) \u{2297} #{three_words[2].show}"
      bra2.each do |hh|
        hh[:coeff] *= h[:coeff]
        total << hh
      end
    end
    three_words.rotate!
  end
  result = total.show(2)
  puts three_words.map(&:show).join(" \u{2297} ") + "  |--> " + result if result != '(0)'
end; nil

## Involutivity for $\nabla$ and $\delta$

$$\nabla\circ \delta = 0$$

In [618]:
length = 8
myw = word_generator(length) #Word.new('ffA')
#puts myw.show + "\n======"

cobra = cobracket(myw)
puts "\u{03B4}(#{myw.show}) = " + cobra.show(1)
puts "---"

total = cobra.each_with_object(Expansion.new) do |h, e|
  coeff = h[:coeff]
  bra = bracket(*h[:words])
  bra.each do |mono|
    mono[:coeff] *= coeff
    e << mono
  end
  #--- display ---
  tensor = h[:words].map(&:show).join(" \u{2297} ")
  puts "\t\u{2207}(#{tensor}) = #{bra.show(1)}"
  #---------------
end

puts "-----\n= " + total.show(1) + "  =  " + total.show(2)
nil

δ(aBefDcdE) = aBDcdE∧ef + (-1)aBE∧efDcd + (-1)aB∧efDcdE + aBeE∧fDcd + aBefcdE∧D + aBef∧DcdE + aBefDdE∧c + aBefDE∧cd + (-1)aBefDcd∧E
---
	∇(aBDcdE ⊗ ef) = (-1)EaBDcd.fe
	∇(aBE ⊗ efDcd) = EaB.fDcde + aBE.Dcdef + (-1)aBE.efDcd
	∇(aB ⊗ efDcdE) = 0
	∇(aBeE ⊗ fDcd) = eEaB.Dcdf + (-1)eEaB.fDcd + (-1)aBeE.Dcdf + aBeE.fDcd
	∇(aBefcdE ⊗ D) = (-1)dEaBefc.D
	∇(aBef ⊗ DcdE) = (-1)efaB.EDcd + faBe.EDcd + aBef.EDcd
	∇(aBefDdE ⊗ c) = DdEaBef.c + (-1)EaBefDd.c
	∇(aBefDE ⊗ cd) = EaBefD.dc
	∇(aBefDcd ⊗ E) = efDcdaB.E + (-1)fDcdaBe.E + (-1)DcdaBef.E
-----
= (-1)EaBDcd.fe + EaB.fDcde + aBE.Dcdef + (-1)aBE.efDcd + eEaB.Dcdf + (-1)eEaB.fDcd + (-1)aBeE.Dcdf + aBeE.fDcd + (-1)dEaBefc.D + (-1)efaB.EDcd + faBe.EDcd + aBef.EDcd + DdEaBef.c + (-1)EaBefDd.c + EaBefD.dc + efDcdaB.E + (-1)fDcdaBe.E + (-1)DcdaBef.E  =  (0)


In [605]:
count = 100 #10000
length = 5 #10
count.times do |k|
  myw = word_generator(length)
  total = cobracket(myw).each_with_object(Expansion.new) do |h, expn|
    bracket(*h[:words]).each do |mono|
      mono[:coeff] *= h[:coeff]
      expn << mono
    end
  end
  result = total.show(2)
  p myw.show + " |--> " + result if result != "(0)"
end; nil

## Compatibility condition for $\nabla$ and $\delta$

$\forall v,w \in \pi$,
$$
\delta([v,w]) = w\cdot\delta(v)- v\cdot\delta(w),
$$
where 
$$
u\cdot(x\otimes y) = [u,x]\otimes y + x\otimes[u,y].
$$
**Note**: From the last formula, we immediately have the following:
$$
u \cdot(x\wedge y) = [u,x]\wedge y - [u,y]\wedge x.
$$

In [634]:
v, w = word_generator(3), word_generator(5) #samples[:s2][0] #Word.new(a[1], a[2]),samples[:s4][0] #Word.new(b[1])
puts "#{v} \u{2297} #{w}"

#--- Left-hand eq. -------------------
lhe = Expansion.new
bra = bracket(v, w)#; puts "\u{2207}(#{v}, #{w}) = " + bra.show
bra.each do |trm|
  cobra = cobracket(trm[:words][0])
#   puts "\t(#{trm[:coeff]})\u{03B4}(#{trm[:words][0].show}) = " + cobra.show(1) if trm[:coeff] != 0
  cobra.each do |mono|
    mono[:coeff] *= trm[:coeff]
#     puts "\t\t(#{mono[:coeff]})#{mono[:words].map(&:show).join(" \u{2227} ")}" if mono[:coeff] != 0
    lhe << mono
  end
end
printf "\nLeft-hand Eq.:  " + lhe.show(2) + "\n---"

#--- Right-hand eq. -------------------
rhe = Expansion.new
ijs = [0,1].product([0,1])
ijs.each do |i,j|
  pair01, sign01 = [v,w].rotate(i), (-1)**i
  #---
  u = pair01[0]
  cobra = cobracket(pair01[1])
  #printf "#{u}.\u{03B4}(#{pair01[1].show}) = #{u}.(#{cobra.show(1)})\n---\n"
  cobra.each do |tm|
    pair02, sign02 = tm[:words].rotate(j), (-1)**j
    #---
    x, y = pair02[0], pair02[1]
    bra = bracket(u, x)
    #; printf "\t[#{u}, #{x}] \u{2297} #{y} \n" + "\t=(#{bra.show(1)}) \u{2297} #{y}"
    bra.each do |mono|
      term = Term.new
      term[:coeff] = mono[:coeff]*tm[:coeff]*sign01*sign02
      term[:words] = [mono[:words][0], y]
      term[:divs] = mono[:divs]
      rhe << term
    end
  end
#   printf "\n---\n"
end
#-----
printf "\nRight-hand Eq.:  "+ rhe.show(2)

#-----------------------
rhe.each do |t|
  t[:coeff] *= -1
  lhe << t
end
lhe.show(2)

BEf ⊗ afDDc

Left-hand Eq.:  BEfafDD∧c + BEfafD∧Dc + BEfafDc∧D + BEfaf∧DDc + BEfafc∧DD + (-1)BEfa∧fDDc + BEffDDc∧a + (-1)BEafDDc∧f + (-1)BfafDDc∧E + (-1)BEfDDca∧f + BEDDcaf∧f + BEffDDa∧c + BEffDa∧Dc + BEffDca∧D + BEffa∧DDc + BEffca∧DD + (-1)BfDDca∧Ef + (-1)BffDDca∧E + (-1)fBEDDcf∧a + (-1)fBEDDaf∧c + (-1)fBEDaf∧Dc + (-1)fBEDcaf∧D + (-1)fBEaf∧DDc + (-1)fBEcaf∧DD + fB∧EDDcaf + (-1)fEDDcaf∧B + (-1)EfBafDD∧c + (-1)EfBafD∧Dc + (-1)EfBafDc∧D + (-1)EfBaf∧DDc + (-1)EfBafc∧DD + (-1)EfBfDDc∧a + (-1)E∧fBafDDc + EBafDDc∧f
---
Right-hand Eq.:  (-1)DDcaffE∧B + (-1)DDcafE∧Bf + (-1)afDDcBE∧f + afDDcEB∧f + DDcafBE∧f + (-1)fDDcaBE∧f + (-1)fDDcaB∧Ef + (-1)afDDcBf∧E + afDDcfB∧E + (-1)fDDcaBf∧E + BEffDDc∧a + (-1)fBEDDcf∧a + (-1)EfBfDDc∧a + BEfafDD∧c + BEffDDa∧c + (-1)fBEDDaf∧c + (-1)EfBafDD∧c + BEfafD∧Dc + BEffDa∧Dc + (-1)fBEDaf∧Dc + (-1)EfBafD∧Dc + BEfafDc∧D + BEffDca∧D + (-1)fBEDcaf∧D + (-1)EfBafDc∧D + BEffa∧DDc + (-1)fBEaf∧DDc + fBEfa∧DDc + (-1)EfBaf∧DDc + BEfafc∧DD + BEffca∧DD + (-1)fBEcaf∧DD + (-1)EfBa

"(0)"

An example whose results of the both side of the compatibility condition are very long.

In [579]:
v,w = Word.new('fbC'), Word.new('CfDBe')
nil

# Experiments

cobracket calclation samples

In [637]:
a1, b1 = Word.new(a[1]), Word.new(b[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    Word.new,
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^2,
    a1^3,
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse),
    a1*(comms[2]^2)*Group.commutator(comms[2].inverse,c)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2]),
    a1*(a1.inverse)*Word.new(b[1])*a1,
    c*(a1.inverse)*a1,
    c*(c.inverse),
    Word.new(a[1], b[1], b[1].inverse, b[2])
  ],
  s7: [
    a1*(Group.conjugate(a1, b1.inverse))*a1*(b1.inverse),
    a1*b1*((a1*(b1.inverse))^2),
    a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse),
    (a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse)).contract,
    a1*a1*(Word.new(b[2].inverse)^2)
    ]
  }

arr = samples #.slice(:s7) #:s0, :s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each{|s| puts s.show + "  |--\u{03B4}-->  " + cobracket(s).show(2) }
end;nil

--- s0 ---
 1  |--δ-->  (0)
a  |--δ-->  (0)
a.d  |--δ-->  (-1)a∧d
a.D  |--δ-->  (0)
a.abA.aBA  |--δ-->  1∧a
--- s1 ---
 a.a  |--δ-->  a∧a
a.a.a  |--δ-->  (0)
a.a.a.a  |--δ-->  aa∧aa
abA  |--δ-->  1∧b
abAB  |--δ-->  (0)
b.a.b  |--δ-->  (0)
--- s2 ---
 a.dcDC  |--δ-->  (-1)a∧dcDC
a.aBA.dcDC.abA  |--δ-->  1∧aBAdcDCab
a.dcDC.cdCDaBAdcDCabA  |--δ-->  (2)1∧aBAdcDCab
aBAdcDCab  |--δ-->  (0)
--- s3 ---
 a.cdCD  |--δ-->  (0)
a.cdCD.cdCD  |--δ-->  acdCD∧cdCD
a.cdCD.abA.cdCD.aBA  |--δ-->  (0)
a.cdCD.cdCD.dcDCabAcdCDaBA  |--δ-->  (0)
--- s4 ---
 a.c.c.c  |--δ-->  acc∧c + ac∧cc
a.cD.cD.cD  |--δ-->  acDcD∧cD + acD∧cDcD
--- s5 ---
 a.ceCE  |--δ-->  (-1)aceC∧E + ace∧CE + (-1)ac∧C
a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  (-1)acdCDefEFceCfeFEdcDC∧E + acdCDefEFcefeFEdcDC∧CE + (-1)acdCDefEFcfeFEdcDC∧C
--- s6 ---
 a.bcBC  |--δ-->  (-1)abcB∧C + abc∧BC + (-1)abC∧cB
a.A.b.a  |--δ-->  (0)
abA.A.a  |--δ-->  1∧b
abA.aBA  |--δ-->  (0)
abBd  |--δ-->  (-1)a∧d + (-1)ad∧1
--- s7 ---
 a.Bab.a.B  |--δ-->  (0)
a.b.a.B.a.B  |

# TODO

* [x] Expansion#show のオプションに "係数 0 の項だけ消去" を加えたい。
* [x] Compatibility が全然ダメ。
* [x] aaa などの Word について、動作が少し変。
* [x] Extension class のメソッドとして、係数 0 を消去したり、共通項で整理したりできるようにしたい。
* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。